### Install libs

In [ ]:
pip install pandas
pip install python-binance
pip install sqlalchemy
pip install backtesting

### Import libs for working with data

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from binance.client import Client 

#create client for getting data and create connection to db
client = Client()
engine = create_engine('sqlite:///test_stocks.db')

### Import libs for testing strategy

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import timedelta

### Initialize function to getting data from binance
Create function, list of tickers, period and get data. Then write all in DB

In [ ]:
def get_data(stock, start):
    frame = pd.DataFrame(client.get_historical_klines(
        stock, 
        '1m', 
        start))
    frame = frame[[0,1,2,3,4]]
    frame.columns = ['Date','Open','High','Low','Close']
    frame.Date = pd.to_datetime(frame.Date, unit='ms')
    frame.set_index('Date', inplace=True)
    frame = frame.astype(float)
    return frame

In [ ]:
#list of stocks
stoks_list = ['BTCUSDT','SOLUSDT','EGLDUSDT',
              'DOGEUSDT','ETHUSDT','XRPUSDT','BNBUSDT']

In [ ]:
#get data for all stocks in list and write it in DB
for stock in stoks_list:
    df = get_data(stock,'80 days ago UTC')
    df.to_sql(stock,engine)

In [ ]:
#Check database thats all correct
pd.read_sql("""SELECT name from sqlite_schema where type='table'""",engine)

### Trading Strategy
Get two SMA lines - 50 and 100 lines of data. If SMA1 cross SMA2, buy position with 3% stop loss and 4% take profit

In [ ]:
class SMACross(Strategy):
    n1 = 50
    n2 = 100
    
    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)
    
    def next(self):
        price = self.data.Close
        if crossover(self.sma1, self.sma2):
            sl = price - price*0.03
            tp = price + price*0.04
            self.buy(sl = sl, tp = tp)

Change dataframe interval. Original data is __1 minute row__. resampleOHLS conver it to new choosed interval. OHLC form is Backtesting library requirement

In [ ]:
def resampleOHLC(df, interval):
    df = df.resample(interval).agg({
        'Open':'first',
        'High':'max',
        'Low':'min',
        'Close':'last'
    })
    return df

Backtesting function. For each stock in list get data from start to date 30 days ago today, convert it in OHLC format with datetime index and make backtesting with parameters. Exclusive order mean that we have only one order at time, new order automatically close previos order. __run__ method returns a pandas Series of simulation results and statistics associated with our strategy.

In [ ]:
def BT(interval):
    res = []
    for stock in stoks_list:
        query = f"""SELECT * from '{stock}'
        where Date < '{pd.to_datetime('today') - timedelta(days = 30)}'"""
        df = pd.read_sql(query,engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df,interval)
        bt = Backtest(df, SMACross, cash = 10000, commission = 0.0015, exclusive_orders = True)
        output = bt.run()
        res.append(output['Return [%]'])
    frame  = pd.DataFrame(res, index=stoks_list, columns=['ret'])
    top = frame.nlargest(3,'ret')
    return top, interval
        

__validate__ function is similar backtesting, but get data from 30 days ago by today

In [ ]:
def validate(top, interval):
    res = []
    for stock in top.index:
        query = f"""SELECT * from '{stock}'
        where Date > '{pd.to_datetime('today') - timedelta(days = 30)}'"""
        df = pd.read_sql(query,engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df,interval)
        bt = Backtest(df, SMACross, cash = 10000, commission = 0.0015, exclusive_orders = True)
        output = bt.run()
        res.append(output['Return [%]'])
    outcome = pd.DataFrame(res, index = top.index, columns=['ret'])
    return outcome

Then get top 3 stocks from backtesting data and check it with validate function on next period. Return mean result of strategy 

In [ ]:
def run(interval):
    bt_result, interval = BT(interval)
    valid = validate(bt_result, interval)
    return valid.ret.mean()

With this function we generate different intervals for calculate SMA and print mean values from top 3 backtesting stocks for this interval

In [ ]:
scale = 'min'
for i in range(10,75,5):
    print(f'Return for {i}{scale}')
    print(run(f'{i}{scale}'))